In [1]:
import requests
from bs4 import BeautifulSoup
import re
from unidecode import unidecode
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import time
import urllib.request
import io
from PIL import Image
import hashlib
import os

In [2]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [3]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [4]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [9]:
chromedriver = '/Users/alex/downloads/chromedriver 2'

abs_path = '/Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images'
os.chdir(abs_path)
for folder in os.listdir()[3:]:
    search_and_download(search_term=folder+' jersey', driver_path=chromedriver, target_path=abs_path+'/'+folder, 
                        number_images=100)

Found: 100 search results. Extracting links from 0:100
Found: 100 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTmqdaP4Cgr2acfA2u0o3BgN8AB50rqYHkrAQ&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Carolina Hurricanes/carolina_hurricanes_jersey/6c6828be88.jpg
SUCCESS - saved https://cdn0.vox-cdn.com/thumbor/MkcZyGxcQZMb2IhP-bLZ77GJ4XE=/250x250/cdn2.vox-cdn.com/uploads/chorus_asset/file/8722215/476339.jpg - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Carolina Hurricanes/carolina_hurricanes_jersey/136d0f9d05.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS9cv8j0f_6frXHCvN2g63RHFUs8Tw77XXAdQ&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Carolina Hurricanes/carolina_hurricanes_jersey/f863a05bca.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=

SUCCESS - saved https://i.ebayimg.com/images/g/sZwAAOSwndFeJHOS/s-l300.jpg - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Carolina Hurricanes/carolina_hurricanes_jersey/6230dcee20.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRlmQsXNWRKCUYaByfgwRAMN8QH_fHRSyH-gw&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Carolina Hurricanes/carolina_hurricanes_jersey/db009608d1.jpg
SUCCESS - saved https://s3951.pcdn.co/wp-content/uploads/2015/09/Carolina-Hurricanes-2000-07-Jerseys.png - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Carolina Hurricanes/carolina_hurricanes_jersey/723d39ae69.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTeE56JpcSmhmCmjIqiryjoNBR3fVvR2Oyw4w&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Carolina Hurricane

SUCCESS - saved https://cdn.shopify.com/s/files/1/1863/2815/products/24_KGrHqUOKpIE5V-sCIhUBOZE_nns0_60_1.jpg?v=1512874067 - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Carolina Hurricanes/carolina_hurricanes_jersey/44f935763e.jpg
SUCCESS - saved https://images.sidelineswap.com/production/001/089/889/feb80ace54044904_small.jpeg - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Carolina Hurricanes/carolina_hurricanes_jersey/2cd0d6a057.jpg
SUCCESS - saved https://s.yimg.com/aah/sports-k/carolina-hurricanes-adidas-nhl-men-s-adizero-authentic-pro-alternate-black-3.gif - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Carolina Hurricanes/carolina_hurricanes_jersey/f5e485f51c.jpg
SUCCESS - saved https://i.pinimg.com/originals/9f/f4/49/9ff449241f2f764625ba3b6ca9d90f8d.jpg - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jers

SUCCESS - saved https://content.sportslogos.net/news/2018/06/Carolina-Hurricanes-Uniforms-2018-19.jpg - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Carolina Hurricanes/carolina_hurricanes_jersey/0b223653b4.jpg
SUCCESS - saved https://www.oursportscentral.com/graphics/pictures/lg20180709-50577.jpg - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Carolina Hurricanes/carolina_hurricanes_jersey/1e08244d63.jpg
SUCCESS - saved https://images.footballfanatics.com/FFImage/thumb.aspx?i=/productimages/_3625000/altimages/ff_3625561-c4b8861f56015a41f413alt1_full.jpg&w=900 - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Carolina Hurricanes/carolina_hurricanes_jersey/f9d693a9b2.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTT6Iq5eKKg4qLnXwJTMu49wu3TYfWQ-fN1fQ&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTTfoFpKlwVfmAdpkPLe7RTWrxhs6d9KI3Rng&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Florida Panthers/florida_panthers_jersey/7e5ed653bf.jpg
SUCCESS - saved https://images-na.ssl-images-amazon.com/images/I/81HupqtViFL._AC_SX425_.jpg - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Florida Panthers/florida_panthers_jersey/94be0bc1db.jpg
SUCCESS - saved https://slimages.macysassets.com/is/image/MCY/products/1/optimized/9240311_fpx.tif?op_sharpen=1&wid=500&hei=613&fit=fit,1&$filtersm$ - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Florida Panthers/florida_panthers_jersey/3c03161041.jpg
SUCCESS - saved https://s3951.pcdn.co/wp-content/uploads/2015/09/Florida-Panthers-2017-Present-Jerseys.png - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_

...

SUCCESS - saved https://1.bp.blogspot.com/-xXXGA7Rc4zc/XdQAUfVBNTI/AAAAAAAAGhQ/4e5XlCluZXceFyNgdPi0z7d3fI7_FBjgQCLcBGAsYHQ/s1600/Dallas%2BStars.jpg - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Dallas Stars/dallas_stars_jersey/3369ad1ea1.jpg
SUCCESS - saved https://www.rallyhouse.com/assets/images/products/14858103-92-STARS-JERSEYS-SPD.jpg - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Dallas Stars/dallas_stars_jersey/71a3a98bee.jpg
SUCCESS - saved https://external-preview.redd.it/EzZ3U_ZY3khM1DhdjyLLpRgZ0j-StnbF63Zrxw6Cnko.png?auto=webp&s=1bfece094979c319cc3c72a9ddb4824f081011f5 - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Dallas Stars/dallas_stars_jersey/0b77aa05f9.jpg
SUCCESS - saved https://static.wixstatic.com/media/39ceee_1d81edc7463e460a8908a53c0b60f930~mv2.jpeg/v1/fill/w_480,h_334,al_c,q_85,usm_0.66_1.00_0.01/39ceee_1d81edc7463e460a8

SUCCESS - saved https://cdn.shopify.com/s/files/1/2725/6498/products/57_7566b54c-0276-48bc-8845-5e388cd2428a.jpg?v=1575450474 - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Dallas Stars/dallas_stars_jersey/98abccd727.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQJI8Su4XxHGXQH7PIBIv9mOH5xLmqBxsrt3g&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Dallas Stars/dallas_stars_jersey/fd4bfb90fd.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTyd7k0M6_9ZN_FSPk4WK1LQif5YDrLKGWfuQ&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Dallas Stars/dallas_stars_jersey/fe14b078ff.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRK3EpP7b4p-GfFUEaUfNg2-I6AIaU9BQHe1Q&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_ima

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcShbTxMij6zI2dLG6scndg6yq8R_ReOipd1Vg&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/New York Rangers/new_york_rangers_jersey/a3a4a2654d.jpg
SUCCESS - saved https://cdn.shopify.com/s/files/1/0020/1025/1324/products/rangers_baseball_tee_1024x1024.jpg?v=1566791139 - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/New York Rangers/new_york_rangers_jersey/fd7332ba86.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSSoQ0cexjhC0EBI_df1fObz_8CsPVYk_va0A&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/New York Rangers/new_york_rangers_jersey/ff77a7bfb4.jpg
SUCCESS - saved https://assets.adidas.com/images/w_600,f_auto,q_auto/e54cc39880d645f7b2dea808000f0a90_9366/Rangers_Home_Authentic_Pro_Jersey_Multicolor_CA6981.jpg - as /Users/alex/desktop/p

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSUP1w-L9RO6eDnZZ-nZQo_9BgrMKP4NOJmBw&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/New York Rangers/new_york_rangers_jersey/20fb1e970a.jpg
SUCCESS - saved https://elitesportsny.com/wp-content/uploads/2019/07/New-York-Rangers-1926-Jerseys-400x208.png - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/New York Rangers/new_york_rangers_jersey/390cb0e217.jpg
SUCCESS - saved https://images.sidelineswap.com/production/006/317/876/e190caa898022440_small.jpeg - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/New York Rangers/new_york_rangers_jersey/df48f8a1a9.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQBYvgdW_ZEznAfsAVCaaGNpktQE3XLrTVdVQ&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/New Yo

SUCCESS - saved https://cdn.shopify.com/s/files/1/1353/0647/products/wayne-gretzky-signed-new-york-rangers-jersey-82157_1024x1024.jpg?v=1530253800 - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/New York Rangers/new_york_rangers_jersey/6a515730e0.jpg
SUCCESS - saved https://www.gamewornauctions.net/images/products/858de0735c6c2c63.jpg - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/New York Rangers/new_york_rangers_jersey/927243f282.jpg
SUCCESS - saved https://d1w8cc2yygc27j.cloudfront.net/6769676608109276965/-9037411748167570435.jpg - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/New York Rangers/new_york_rangers_jersey/fd528182af.jpg
SUCCESS - saved https://nhl.bamcontent.com/images/photos/290027112/1024x576/cut.jpg - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/New York Rangers/new_york_rangers_je

In [10]:
search_and_download(search_term='Washington Capitals jersey', driver_path=chromedriver, 
                    target_path=abs_path+'/'+'Washington Capitals', number_images=100)

Found: 100 search results. Extracting links from 0:100
Found: 101 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSO9FZ5Xgty92TsIagVxkwenA7NRVZb1VgvWw&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Washington Capitals/washington_capitals_jersey/29ff1e1cf9.jpg
SUCCESS - saved https://s.yimg.com/aah/homegrownsportinggoods/athletic-knit-washington-capitals-knit-hockey-jerseys-41.gif - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Washington Capitals/washington_capitals_jersey/cd6b4c423e.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQs85lc7bahqXL7_MRf51g_U5gCOtbc2a4iTA&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Washington Capitals/washington_capitals_jersey/80e92f0389.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSJhzsDz-zoiI5

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRuPHxa4gVS-TFqyTjOGT8MaWKCzT5TSp2_ig&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Washington Capitals/washington_capitals_jersey/ab84f3fc38.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQdlTwYyMW-99mC4BZecr9ZDhBG5Im2nbZ4jg&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Washington Capitals/washington_capitals_jersey/8b808108f1.jpg
SUCCESS - saved https://i.pinimg.com/originals/40/31/44/403144ed9dcab7167a8c8b2856df0a3f.jpg - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Washington Capitals/washington_capitals_jersey/e8b580ea4d.jpg
SUCCESS - saved https://d9nvuahg4xykp.cloudfront.net/-5543374152191413564/-2091918656512991595.jpg - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Washin

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTcFxLuD3WEyOvCNMnxSajS0CkfhdcpRmHngA&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Washington Capitals/washington_capitals_jersey/261960f55b.jpg
SUCCESS - saved https://images.sidelineswap.com/production/007/089/965/7ba18cefb0f8c7aa_small.jpeg - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Washington Capitals/washington_capitals_jersey/b213bee65e.jpg
SUCCESS - saved https://previews.123rf.com/images/grebeshkovmaxim/grebeshkovmaxim1902/grebeshkovmaxim190200379/125226959-realistic-sport-shirt-washington-capitals-jersey-template-for-ice-hockey-kit-vector-illustration.jpg - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Washington Capitals/washington_capitals_jersey/8ca6e2420e.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS5yHfUGyPhkHg2gosu

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTaFOuEzkV6K3UVwjkoVAr1Lm1DvWeqIxqnmA&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Washington Capitals/washington_capitals_jersey/533f66ef11.jpg
SUCCESS - saved https://i1.wp.com/novacapsfans.com/wp-content/uploads/2017/08/img_9954.jpg?resize=640%2C423&ssl=1 - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Washington Capitals/washington_capitals_jersey/d848c24d0f.jpg
SUCCESS - saved https://ik.imagekit.io/r16kltudm/game7/pub/media/tr:w-299,h-299,cm-pad_resize,bg-FFFFFF/catalog/product//d/s/dsc_6252.jpg - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Washington Capitals/washington_capitals_jersey/44511f7709.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQNxshjjTjDDpJ_Hci1lDX-PgzTl_NXTKFUSA&usqp=CAU - as /Users/alex/desktop/python_work/pyth

In [11]:
search_and_download(search_term='Edmonton Oilers jersey', driver_path=chromedriver, 
                    target_path=abs_path+'/'+'Edmonton Oilers', number_images=100)

Found: 100 search results. Extracting links from 0:100
Found: 100 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQeMxU6xkE3UhiMHI_Dk9mEhNBOZwMIHw8-1A&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Edmonton Oilers/edmonton_oilers_jersey/29c8b33a4f.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTlyEd4ORHrHrgZQ10slWtp86O9doUHf547uQ&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Edmonton Oilers/edmonton_oilers_jersey/aa2b2c037f.jpg
SUCCESS - saved https://sep.yimg.com/ay/sports-k/vintage-edmonton-oilers-road-blue-jersey-1981-1996-7.gif - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Edmonton Oilers/edmonton_oilers_jersey/8888486677.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQx5DyCHR-EG9nElYMCKZSPCbrHoGY6kYIxaQ&usqp=CAU - as /Us

SUCCESS - saved https://www.coolhockey.com/media/catalog/product/cache/efbb07c69433f77a59889e99956d8466/o/i/oilers_f.jpg - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Edmonton Oilers/edmonton_oilers_jersey/aac2aea9ac.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQxJS20QhiO4AXCaxV2Z0P1X5l_tGw1q4ZuKA&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Edmonton Oilers/edmonton_oilers_jersey/4278de2e51.jpg
SUCCESS - saved https://fanatics.frgimages.com/FFImage/thumb.aspx?i=/productimages/_3276000/altimages/ff_3276558-f13c3a2e70da5b15667calt1_full.jpg&w=900 - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Edmonton Oilers/edmonton_oilers_jersey/2d4cbde2ed.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTu2i6lT3k_0u5kZBm2PbCkF44yturSxP3enw&usqp=CAU - as /Users/alex/desktop/python_work/python

SUCCESS - saved https://images.sidelineswap.com/production/002/848/221/665ab3a72514dd49_small.jpeg - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Edmonton Oilers/edmonton_oilers_jersey/4a1733dca0.jpg
SUCCESS - saved https://i.imgur.com/GuOUKvB.jpg - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Edmonton Oilers/edmonton_oilers_jersey/a4e3c1b61d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSQxf-pv1EHgtTNQNAMBGbcH9CV8AYcw-kLFQ&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Edmonton Oilers/edmonton_oilers_jersey/cc26cde88c.jpg
SUCCESS - saved https://www.coolhockey.com/media/catalog/product/cache/efbb07c69433f77a59889e99956d8466/g/r/gretzky_1.jpg - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Edmonton Oilers/edmonton_oilers_jersey/36a63be037.jpg
SUCCESS - saved h

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQubm7iAFi-Nfukl8L0ebOArNNpjPiBRf3AWA&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Edmonton Oilers/edmonton_oilers_jersey/2599607b0d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcT-oZJikfTHV0sWTj7RStkt0V_z4o93ou5LaQ&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Edmonton Oilers/edmonton_oilers_jersey/a4be403d14.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ0QxZssOHLAx25UE29hIBOOQV5zRrMVmEYEA&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_jersey_images/Edmonton Oilers/edmonton_oilers_jersey/478b39bab7.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQOH_3NVeQCSkV8KVSs9iTGnO1hqKYXHW8gCw&usqp=CAU - as /Users/alex/desktop/python_work/python_projects/NHL_Jersey_Classifiers/team_